In [3]:
import rasterio as rio
import numpy as np
from glob import glob
import gdal
import ogr
import numpy.ma as ma

from rasterio.mask import mask


####SCRIPT FOR COASTLINE DETECTION

# open and read green and NIR bands using rasterio 
#n_src should be the raster for the band covering near infrared wavelengths (NIR, band 5 for Landsat8 products)
#g_src should be the raster for the band covering green wavelengths (band 3 for Landsat8 products)

with rio.open(r"C:\Users\lmari\Documents\GIS\Landsat\clipped\b3_clipped.tif") as g_src:
    green= g_src.read()
    #gets metadata from source file and assigns it to variable.  Use this later for kwargs
    meta=g_src.profile
    
#nir_path should be the raster for the band covering near infrared wavelengths (NIR, band 5 for Landsat8 products)
with rio.open(r"C:\Users\lmari\Documents\GIS\Landsat\clipped\b5_clipped.tif") as n_src:
    nir= n_src.read()


# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

# Calculate NDWI
ndwi = (green.astype(float) - nir.astype(float)) / (nir + green)

#close files
g_src.close()
n_src.close()

    #prints min and max for NDWI to double check bounds

#mx= float(ndwi.max())
#mn= float(ndwi.min())
#print ("Max value for NDWI is: " + str(mx)) 
#print ("Min value for NDWI is: " + str(mn))

    #checks array shape (useful for further processing and writing to files later)

#print ("NDWI shape is: " + str(ndwi.shape))

#####Print NDWI Tif

#Formats the numpy array to match the raster formatting for printing to tif.  Removes "1" values from array shape
ndwi_reshape= np.squeeze(ndwi)


#gets metadata from input tiffs to pass to output tiff

kwargs= meta

#updates meta data to float type for rasterio to write to tiff
kwargs.update(dtype= 'float64',
              count=1)

# write the file to a tiff
with rio.open('ndwi1.tif', 'w', **kwargs) as dst:
    dst.write(ndwi_reshape, 1)
    

dst.close()
print ("NDWI Tif has been written")


##### Make Binary Mask
#create the mask by assigning pixel values to land and water based on NDWI values 
#NDWI values
# <0 is land
# >=0 is water 


#iterates through the NDWI array and sets all values <0 == 0 and all values >= 0 == 1
#0 represents land
#1 represents water 

with np.nditer(ndwi_reshape, op_flags=['readwrite'], op_dtypes=[np.float64]) as it:
    for i in it:
        if i < 0:
            i[...] = 0
        elif i >= 0:
            i[...] = 1
    np.nditer.close(it)

mx= float(ndwi_reshape.max())
mn= float(ndwi_reshape.min())

#casts as float32 d type (necessary for future manipulation with rasterio)
ndwi_reshape.astype(dtype= 'float32')


###Writes Binary Mask TIF


kwargs= meta
ndwi_32=ndwi_reshape.astype(dtype= 'float32')

#updates meta data to float type 
kwargs.update(dtype= 'float32',
              count=1)


# write the file to a tiff
with rio.open('ndwimask.tif', 'w', **kwargs) as dst:
    dst.write(ndwi_32, 1)
    print ('NDWI masked GeoTIFF has been saved')
  
dst.close()


#####Check neighbor pixels to find coast

#supress allows for forced processing when errors are raised 
from contextlib import suppress

#Checks to see if neighboring cells are different.  if so, places a "True" value for those cells
def compare_neighbor(arr):
    
    #Makes a new array with same shape as input and sets all values to false
    comp_arr= np.full(arr.shape, False, dtype=int)
    
    for (x, y), item in np.ndenumerate(arr):
        # Check left.
        if x >= 1:
            if arr[x-1, y] != item:
                comp_arr[x, y] = True
                continue

        # Check right.
        with suppress(IndexError):
            if arr[x+1, y] != item:
                comp_arr[x, y] = True
                continue

        # Check top.
        with suppress(IndexError):
            if arr[x, y+1] != item:
                comp_arr[x, y] = True
                continue

        # Check bottom.
        if y >= 1:
            if arr[x, y-1] != item:
                comp_arr[x, y] = True
                continue

    return comp_arr
#new variable with only those cells that represent border (coast areas) set to 1
coast_mask=compare_neighbor(ndwi_reshape)

#####Print Coast_mask to tif which represents coastline

kwargs= meta

#updates meta data to int32 type 
kwargs.update(dtype= 'int32',
              count=1)

# write the file to a tiff
with rio.open('coastbound.tif', 'w', **kwargs) as dst:
    dst.write(coast_mask, 1)

    print ('coastbound GeoTIFF has been saved')
  
dst.close()

#####Write to shapefile
#Makes Raster pixels of 1 into polygon and writes to shapefile 

kwargs= meta

#  get raster and mask datasource
src_ds = gdal.Open( "ndwi.tif" )
srcband = src_ds.GetRasterBand(1)
maskfile= gdal.Open('coastmask.tif')
mask= maskfile.GetRasterBand(1)

#  create output datasource

dst_layername = "coastline"
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource(dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername, srs = None )

gdal.Polygonize(srcband, mask, dst_layer, -1, [], callback=None )

#Close and reset files to prevent errors
dst_ds.Destroy()
src_ds=None

print("coastline.shp has been saved")


NDWI Tif has been written
NDWI masked GeoTIFF has been saved
coastbound GeoTIFF has been saved
coastline.shp has been saved
